# Proyecto Final - Precios de SUVs 

In [1]:
%load_ext autoreload
%autoreload 2

### Analisis dataset crudo

In [2]:
import pandas as pd
# Cargar el dataset
df_raw = pd.read_csv("dataset/raw/pf_suvs_i302_1s2025.csv")
df_raw = pd.DataFrame(df_raw)

In [3]:
from src.data_exploration import*

In [4]:
exploratory_analysis(df_raw, target_col="Precio")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18254 entries, 0 to 18253
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               18254 non-null  int64  
 1   Marca                    18254 non-null  object 
 2   Modelo                   18254 non-null  object 
 3   Año                      18254 non-null  float64
 4   Versión                  18254 non-null  object 
 5   Color                    17865 non-null  object 
 6   Tipo de combustible      18254 non-null  object 
 7   Puertas                  18254 non-null  float64
 8   Transmisión              18239 non-null  object 
 9   Motor                    18216 non-null  object 
 10  Tipo de carrocería       18254 non-null  object 
 11  Con cámara de retroceso  4691 non-null   object 
 12  Kilómetros               18254 non-null  object 
 13  Título                   18254 non-null  object 
 14  Precio                

In [5]:
df_raw["Moneda"].value_counts()

Moneda
$      11158
US$     7096
Name: count, dtype: int64

In [6]:
df_raw["Marca"].value_counts()

Marca
Ford              2161
Jeep              2050
Volkswagen        2037
Chevrolet         1750
Renault           1491
Toyota            1260
Peugeot           1250
Nissan            1059
Citroën            721
BMW                672
Honda              597
Hyundai            568
Audi               529
Fiat               366
Chery              344
Kia                290
Mercedes-Benz      282
Dodge              185
BAIC               165
Suzuki              76
Porsche             67
Land Rover          65
Mitsubishi          47
Volvo               38
SsangYong           30
D.S.                27
Alfa Romeo          25
JAC                 19
Jetour              14
GWM                  9
Haval                9
Isuzu                8
Lifan                7
Lexus                7
Subaru               5
Daihatsu             4
D·S                  4
Mini                 3
Hiunday              2
KAIYI                2
DS AUTOMOBILES       2
Jaguar               2
Range Rover          1
Rrena

In [7]:
df_raw["Transmisión"].value_counts()

Transmisión
Automática               12281
Manual                    5834
Automática secuencial      123
Semiautomática               1
Name: count, dtype: int64

In [8]:
df_raw["Tipo de vendedor"].value_counts()

Tipo de vendedor
concesionaria    8509
particular       5676
tienda           4069
Name: count, dtype: int64

In [9]:
df_raw["Tipo de carrocería"].value_counts()

Tipo de carrocería
SUV    18254
Name: count, dtype: int64

In [10]:
df_raw["Tipo de combustible"].value_counts()

Tipo de combustible
Nafta            16067
Diésel            1328
Nafta/GNC          451
Híbrido/Nafta      269
Híbrido            105
GNC                 12
Eléctrico           12
Mild Hybrid         10
Name: count, dtype: int64

In [11]:
from src.data_cleaner import DataProcessor

### Pre-procesamientos del dataset

In [12]:
# usar distintos tipos de datasets
dp_OH = DataProcessor(df_raw)
df_OH = pd.DataFrame(dp_OH.preprocess())
df_OH.to_csv("dataset/processed/marcas_OH.csv", index=False)

dp_OH_no_grouping = DataProcessor(df_raw, config={
    "group_transmission": False,
    "group_combustible": False})
df_OH_no_grouping = pd.DataFrame(dp_OH_no_grouping.preprocess())
df_OH_no_grouping.to_csv("dataset/processed/marcas_OH_no_grouping.csv", index=False)

df_no_brands = df_raw.copy()
df_no_brands = df_no_brands.drop(columns=["Marca", "Modelo"], errors="ignore")  
dp_no_brands_grouping = DataProcessor(df_no_brands, config={
    "one_hot_encode": False
})
df_no_brands_grouping = dp_no_brands_grouping.preprocess()
df_no_brands_grouping.to_csv("dataset/processed/sin_marcas_grouping.csv", index=False)


dp_no_brands_no_grouping = DataProcessor(df_no_brands, config={
    "one_hot_encode": False,
    "group_transmission": False,
    "group_combustible": False})
df_no_brands_no_grouping = pd.DataFrame(dp_no_brands_no_grouping.preprocess())
df_no_brands_no_grouping.to_csv("dataset/processed/sin_marcas_no_grouping.csv", index=False)

In [13]:
exploratory_analysis(df_no_brands_grouping, target_col="Precio_usd")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17781 entries, 0 to 17780
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Puertas              17781 non-null  float64
 1   Kilómetros           17781 non-null  float64
 2   Precio_usd           17781 non-null  float64
 3   Antigüedad           17781 non-null  float64
 4   Cilindrada           17781 non-null  float64
 5   Transmisión_Manual   17781 non-null  int64  
 6   vendedor_particular  17781 non-null  int64  
 7   combustible_Diésel   17781 non-null  bool   
 8   combustible_Nafta    17781 non-null  bool   
 9   combustible_Otros    17781 non-null  bool   
dtypes: bool(3), float64(5), int64(2)
memory usage: 1.0 MB
None

--- Resumen Estadístico ---
            Puertas    Kilómetros     Precio_usd     Antigüedad    Cilindrada  \
count  17781.000000  1.778100e+04   17781.000000   17781.000000  17781.000000   
mean      18.522243  7.518157e+05   

In [14]:
from models.linear_regression import LinearReg
from src.train_val_models import*

### Comparar metricas de varios modelos

In [15]:
datasets = {
    "One-hot completo": (dp_OH, df_OH)
}

resultados = run_experiment(datasets)


[One-hot completo] MSE: 354291393.07 | RMSE: 18822.63 | MAE: 11624.33


Probe crear una columna gama pero usando el precio, creo que hay data leak

In [27]:
from src.cross_val import cross_validate_rf, tune_and_test_rf

### Random forest parameters exploration

In [ ]:

import pandas as pd
from sklearn.ensemble import RandomForestRegressor


datasets_explorations = [
    ("One-hot completo", df_OH),
    ("One-hot sin agrupamiento", df_OH_no_grouping),
    ("Sin marcas con agrupamiento", df_no_brands_grouping),
    ("Sin marcas sin agrupamiento", df_no_brands_no_grouping)
]

param_grid = {
    "n_trees": [20, 50,100,200],             
    "max_depth": [5, 20, 50,None],         
    "min_samples_split": [2, 5, 10],        
    "max_features": [5,20, "sqrt", "log2"] 
}
resultados = []

for nombre, dataset in datasets_explorations:
    print(f"\nEvaluando dataset: {nombre}")
    res = tune_and_test_rf(dataset, target_col="Precio_usd", param_grid=param_grid, top_n=3)
    print(res)
    resultados.append({
        "dataset": nombre,
        **res  # si 'res' es un diccionario con los campos de resultados
    })





📊 Evaluando dataset: One-hot completo
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 5} --> RMSE promedio: 16565.2956
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 20} --> RMSE promedio: 13058.9772
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'sqrt'} --> RMSE promedio: 13976.8248
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'log2'} --> RMSE promedio: 15733.0827
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 5} --> RMSE promedio: 16576.6969
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 20} --> RMSE promedio: 13211.9844
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'sqrt'} --> RMSE promedio: 14095.3699
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'log2'} --> RMSE promedio: 15847.7209
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 10, 'max_features': 5} --> RMSE promedio: 1659

In [ ]:
df_resultados = pd.DataFrame(resultados)

for _, row in df_resultados.iterrows():
    print(f"\n📌 Dataset: {row['dataset']}")
    print(f"🔧 Best Val Params:  {row['best_params_val']}")
    print(f"📉 Val RMSE:         {row['val_rmse']:.2f}")
    print(f"🏆 Best Test Params: {row['best_params_test']}")
    print(f"🧪 Test RMSE:        {row['test_rmse']:.2f}")



📌 Dataset: One-hot completo
🔧 Best Val Params:  {'n_trees': 200, 'max_depth': 50, 'min_samples_split': 5, 'max_features': 20}
📉 Val RMSE:         6946.66
🏆 Best Test Params: {'n_trees': 200, 'max_depth': 50, 'min_samples_split': 2, 'max_features': 20}
🧪 Test RMSE:        4793.33

📌 Dataset: One-hot sin agrupamiento
🔧 Best Val Params:  {'n_trees': 100, 'max_depth': None, 'min_samples_split': 2, 'max_features': 20}
📉 Val RMSE:         7052.94
🏆 Best Test Params: {'n_trees': 200, 'max_depth': None, 'min_samples_split': 2, 'max_features': 20}
🧪 Test RMSE:        4839.08

📌 Dataset: Sin marcas con agrupamiento
🔧 Best Val Params:  {'n_trees': 20, 'max_depth': 50, 'min_samples_split': 10, 'max_features': 5}
📉 Val RMSE:         9294.81
🏆 Best Test Params: {'n_trees': 20, 'max_depth': 50, 'min_samples_split': 10, 'max_features': 5}
🧪 Test RMSE:        8357.68

📌 Dataset: Sin marcas sin agrupamiento
🔧 Best Val Params:  {'n_trees': 200, 'max_depth': 20, 'min_samples_split': 10, 'max_features': 2

In [25]:
# Dataset 2: agregando features numéricas derivadas
dp_numeric_feats = DataProcessor(df_raw, config={
    "add_precio_por_km": True,
    "add_antiguedad_squared": True,
    "add_cilindrada_times_km": True,
    "add_frecuencia_features": False
})
df_numeric_feats = dp_numeric_feats.preprocess()
df_numeric_feats.to_csv("dataset/processed/dataset_numeric_feats.csv", index=False)


# Dataset 3: agregando también frecuencia de marcas
dp_all_feats = DataProcessor(df_raw, config={
    "add_precio_por_km": True,
    "add_antiguedad_squared": True,
    "add_cilindrada_times_km": True,
    "add_frecuencia_features": True
})
df_all_feats = dp_all_feats.preprocess()
df_all_feats.to_csv("dataset/processed/dataset_all_feats.csv", index=False)

In [30]:
exploratory_analysis(df_all_feats, target_col="Precio_usd")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17781 entries, 0 to 17780
Columns: 194 entries, Puertas to frecuencia_modelo
dtypes: bool(182), float64(10), int64(2)
memory usage: 4.7 MB
None

--- Resumen Estadístico ---
            Puertas    Kilómetros     Precio_usd     Antigüedad    Cilindrada  \
count  17781.000000  1.778100e+04   17781.000000   17781.000000  17781.000000   
mean      18.522243  7.518157e+05   27390.631641     -18.399753      1.817519   
std      903.547364  1.064227e+06   19624.074249    3259.772123      0.625942   
min        2.000000  0.000000e+00     100.399912 -434669.000000      1.000000   
25%        5.000000  0.000000e+00   16021.716754       1.000000      1.500000   
50%        5.000000  6.800000e+05   23454.769418       6.000000      1.600000   
75%        5.000000  1.160000e+06   32229.216374       9.000000      2.000000   
max    60252.000000  9.999999e+07  610000.000000      39.000000      6.400000   

       Transmisión_Manual  vendedor_particular 

In [ ]:
resultados = []

datasets_explorations = [
    ("One-hot completo", df_OH),
    ("Dataset con features numéricas", df_numeric_feats),
    ("Dataset con features completas", df_all_feats)
]

for nombre, dataset in datasets_explorations:
    print(f"\n📊 Evaluando dataset: {nombre}")
    res = tune_and_test_rf(dataset, target_col="Precio_usd", param_grid=param_grid, top_n=3)
    print(res)
    resultados.append({
    "dataset": nombre,
    "best_params_val": res["best_params_val"],
    "val_rmse": res["val_rmse"],
    "best_params_test": res["best_params_test"],
    "test_rmse": res["test_rmse"]
})






📊 Evaluando dataset: One-hot completo
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 5} --> RMSE promedio: 16565.2956
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 20} --> RMSE promedio: 13058.9772
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'sqrt'} --> RMSE promedio: 13976.8248
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 2, 'max_features': 'log2'} --> RMSE promedio: 15733.0827
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 5} --> RMSE promedio: 16576.6969
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 20} --> RMSE promedio: 13211.9844
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'sqrt'} --> RMSE promedio: 14095.3699
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 5, 'max_features': 'log2'} --> RMSE promedio: 15847.7209
{'n_trees': 20, 'max_depth': 5, 'min_samples_split': 10, 'max_features': 5} --> RMSE promedio: 1659

In [ ]:
# Ahora, para ver resultados ordenados por test_rmse, por ejemplo:
import pandas as pd

df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values(by="test_rmse")
print("\n📈 Ranking de datasets por menor RMSE en test:")
print(df_resultados.to_string(index=False))